Download the trained weights for plitterStreet detections

In [ ]:
# !git clone https://github.com/gicait/pLitter
# !pip install /content/pLitter

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1lRKR5SfjhmaWichmBg2TWPufUVDed4zF' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1lRKR5SfjhmaWichmBg2TWPufUVDed4zF" -O /content/yolov5l.pt && rm -rf /tmp/cookies.txt

In [ ]:
# !mkdir data
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1nHRp1_0oik9fVsY0tmVuyarjuxPdXyDL' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1nHRp1_0oik9fVsY0tmVuyarjuxPdXyDL" -O data/sample.zip && rm -rf /tmp/cookies.txt
# !unzip data/sample.zip -d data/.
# !rm -rf 'data/sample.zip'

GoPro videos not working properly with opencv

use ffmpeg to copy the video track to a new file 


In [ ]:
# %cd 'directory_to_videos'

# !for FILE in *.MP4; do ffmpeg -i $FILE -c:v copy "target_dir_to_save_video_tracks"/${FILE//MP4/mp4} -y; done

In [4]:
import os, cv2, json, time
import torch
# from plitterstreet.predict import draw_boxes_on_image

In [5]:
colors = [(0,255,255), (0,0,255), (255,0,0), (0,255,0)]
def draw_boxes_on_image(image, boxes, classes, class_ids, scores, use_normalized_coordinates=True, min_score_thresh=.3):
    assert len(boxes) == len(scores) == len(classes)
    for i in range(len(boxes)):
        box = boxes[i]
        category = str(classes[i])
        class_id = int(class_ids[i])
        score = scores[i]
        if score >= min_score_thresh:
          if use_normalized_coordinates:
            h,w,_ = image.shape
            y1 = int(box[0]*h)
            x1 = int(box[1]*w)
            y2 = int(box[2]*h)
            x2 = int(box[3]*w)  
          else:
            x1,y1,x2,y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])
          image = cv2.rectangle(image, (x1, y1), (x2, y2), colors[class_id], 2)
          cv2.putText(image, category+':'+str(round(score,2)), (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[class_id], 1)
    return image

In [ ]:
weights = '/content/yolov5l.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', weights, force_reload=True)

In [7]:
# img = cv2.imread('/content/drive/MyDrive/roxana-charles-st-gopro-capture-sept30/GH010017/GH010017_10.jpg')
# preds = model(img, size=1280)
# class_ids = preds.xyxy[0].cpu().numpy()[:, 5]
# classes = [preds.names[cls] for cls in class_ids]
# scores = preds.xyxy[0].cpu().numpy()[:, 4]
# boxes = preds.xyxy[0].cpu().numpy()[:,:4]
# img = draw_boxes_on_image(img, boxes, classes, class_ids, scores, use_normalized_coordinates=False, min_score_thresh=.4)

# from google.colab.patches import cv2_imshow
# cv2_imshow(img)

In [ ]:
#to do
#pandas to any file format
#tracking

In [14]:
source = 'data'
target = 'data'

if source == target:
  print('source should not be same as target, creating run folder insode to save results')
  os.makedirs(source+'/'+'run', exist_ok=True)
  target = source+'/'+'run'
  # assert(False)


if os.path.isfile(source):
  dataset = [os.path.basename(source)]
  source = os.path.dirname(source)
elif os.path.isdir(source):
  dataset = sorted([im for im in os.listdir(source) if im.lower().endswith(('.mp4', '.avi', '.jpg', '.png'))])
else:
  print('please check source is correct')

print(dataset)

for fn in dataset:
  print(fn)
  if fn.endswith(('.jpg', '.png')):
    img = cv2.imread(os.path.join(source, fn))
    preds = model(img, size=1280)
    class_ids = preds.xyxy[0].cpu().numpy()[:, 5]
    classes = [preds.names[cls] for cls in class_ids]
    scores = preds.xyxy[0].cpu().numpy()[:, 4]
    boxes = preds.xyxy[0].cpu().numpy()[:,:4]
    img = draw_boxes_on_image(img, boxes, classes, class_ids, scores, use_normalized_coordinates=False, min_score_thresh=.4)
    cv2.imwrite(os.path.join(target, fn), img)
    pred_json = {'file_name': fn, 'category_ids': class_ids, 'boxes': boxes, 'scores': scores}
    json.dump(pred_json, os.path.join(target, fn.replace(('.jpg', '.png'), '.json')))
  else:
    st = time.time()
    vid = cv2.VideoCapture(os.path.join(source, fn))
    width  = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(vid.get(cv2.CAP_PROP_FPS))
    # print(width, height)
    # fn_r = os.path.join(target, fn.split('.')[0]+'.avi')
    fn_r = os.path.join(target, fn)
    print(fn_r)
    # result = cv2.VideoWriter(fn_r, cv2.VideoWriter_fourcc(*'MJPG'), fps, (width, height))
    result = cv2.VideoWriter(fn_r, cv2.VideoWriter_fourcc(*'MP4V'), fps, (width, height))
    # result = cv2.VideoWriter(fn_r, cv2.VideoWriter_fourcc(*'H264'), fps, (width, height))
    pred_json = {'video': fn, 'preds': []}
    c = 1
    while True:
        ret, img = vid.read()
        if img is None:
          json.dump(pred_json, open(os.path.splitext(fn_r)[0]+'.json', 'w'))
          result.release()
          vid.release()
          break
        preds = model(img, size=1280)
        class_ids = preds.xyxy[0].cpu().numpy()[:, 5]
        classes = [preds.names[cls] for cls in class_ids]
        scores = preds.xyxy[0].cpu().numpy()[:, 4]
        boxes = preds.xyxy[0].cpu().numpy()[:,:4]
        im = draw_boxes_on_image(img, boxes, classes, class_ids, scores, use_normalized_coordinates=False, min_score_thresh=.4)
        # cv2.imwrite('/content/frames/'+str(c)+'.jpg', im)
        pred_json['preds'].append({'frame': c, 'category_ids': class_ids.tolist(), 'boxes': boxes.tolist(), 'scores': scores.tolist()})
        # print(img.shape)
        result.write(im)
        c+=1
    print(time.time()-st)

source should not be same as target, creating run folder insode to save results
['sample_video.mp4']
sample_video.mp4
data/run/sample_video.mp4
55.39088177680969
